2. Medical Device Company A comes to us and wants to find out how many patients with diabetes are under 75, have the following diagnostic codes: 408850009, 232063007, 232053004, a total Cholesterol reading between 185 and 230, and a diastolic blood pressure reading of over 100. How many patients meet this criteria? How would you report this information back to Medical Device Company A?

In [ ]:
-- Question 2 SQL answer
select count(*) as patient_count
from patient_clinical pc
join patient_demographics pd ON pc.patient_id=pd.patient_id 
join patient_history ph ON pc.patient_id=pd.patient_id
where pc.diagnosis_code in ('408850009', '232063007', '232053004')
AND pc.TOTCHOL BETWEEN 185 and 230
AND pc.DIABP > 100 
AND pd.age < 75 
AND ph.diabetes = 1;


In [ ]:
# Question 3 Python answer
import pandas as pd

def open_and_clean(csv_file):
    # open csv file as a dataframe
    heart_df = pd.read_csv(csv_file)

    # set column headers to lower for consistency
    heart_df.columns = heart_df.columns.str.lower()

    # drop columns with all null values, keep patient_id and note columns
    heart_df.dropna(axis=1, how='all', inplace=True)

    return heart_df

# function to search for keywords in csv
def contains_keyword(note):
    keywords = [
    'heart attack', 'pain in chest', 'pain in the chest', 'fluttering', 
    'pressure in chest', 'tightness in chest', 'chest pain', 'pressure in the chest'
]
    note = note.lower()
    return any(keyword in note for keyword in keywords)

def main():
    # user defined file path 
    csv_file = '/Users/carrieminerich/Desktop/verana/patient_note.csv'

    # run open_and_clean function
    heart_df = open_and_clean(csv_file)

    # create final_heart_df by including only keywords from heart_df
    final_heart_df = heart_df[heart_df['note'].apply(contains_keyword)]

    # count of rows in final_heart_df
    print(f"Count of rows in final_heart_df: {len(final_heart_df)}")

    final_heart_df.to_csv('filterd_heart_dataset.csv', index=False) 

if __name__ == "__main__":
    main()

In [ ]:
-- Question 3 SQL answer
SELECT --
  COUNT(DISTINCT patient_id) AS heart_patient_count
FROM
  VERANA.PUBLIC.patient_note
WHERE
  note ILIKE '%heart attack%'
  OR note ILIKE '%pain in chest%'
  OR note ILIKE '%pain in the chest%'
  OR note ILIKE '%fluttering%'
  OR note ILIKE '%pressure in chest%'
  OR note ILIKE '%tightness in chest%'
  OR note ILIKE '%chest pain%' 
  OR note ILIKE '%pressure in the chest%'
  ;



In [ ]:
-- Question 4 SQL answer
WITH heart_patient_count AS (
  SELECT
    patient_id
  FROM
    patient_note
  WHERE
    note ILIKE '%heart attack%'
  OR note ILIKE '%pain in chest%'
  OR note ILIKE '%fluttering%'
  OR note ILIKE '%pressure in chest%'
  OR note ILIKE '%tightness in chest%'
  OR note ILIKE '%chest pain%' 
)
SELECT
  COUNT(DISTINCT pd.patient_id) AS heart_patient_count
FROM heart_patient_count hpc
  JOIN patient_demographics pd ON hpc.patient_id = pd.patient_id
  JOIN patient_clinical pc ON pd.patient_id = pc.patient_id
WHERE
  pd.male = 1
  AND (
    pc.diagnosis_code = '232065000'
    OR pc.diagnosis_code = 'H35.52'
  );



In [ ]:
# Question 5 Python answer
import pandas as pd
from scipy import stats

# open csv and set as dataframe
patient_clinical_df = pd.read_csv('/Users/carrieminerich/Desktop/verana/patient_clinical.csv')
patient_demographics_df = pd.read_csv('/Users/carrieminerich/Desktop/verana/patient_demographics.csv')

# make sure column headers are lower case for easier analysis
patient_clinical_df.columns = patient_clinical_df.columns.str.lower()
patient_demographics_df.columns = patient_demographics_df.columns.str.lower()

# merge dfs on patient_id
merged_df = patient_clinical_df.merge(patient_demographics_df, on='patient_id')

# separate merged_df into men and women df
men_df = merged_df[merged_df['male'] ==1]
women_df = merged_df[merged_df['male'] == 0]

# calculate distribution 
distribution_men = men_df['tenyearchd'].value_counts(normalize=True)
distribution_women = women_df['tenyearchd'].value_counts(normalize=True)

print(f"Distribution of TenYearCHD among men:{distribution_men}")
print(f"Distribution of TenYearCHD among women:{distribution_women}")

# matrix is count of occurance for each value male and tenyearchd
matrix = pd.crosstab(merged_df['male'], merged_df['tenyearchd'])
print(matrix)

chi2, p, _, _ = stats.chi2_contingency(matrix)

print(f"\nChi-Square Test Statistic: {chi2}")
print(f"P-Value: {p}")

# print results
if p < 0.05:
    print("\nThe result is statistically significant. There is an association between gender and TenYearCHD.")
else:
    print("\nThe result is not statistically significant. There is no strong evidence of an association between gender and TenYearCHD.")


In [ ]:
# Question 6 Python answer
import pandas as pd
from scipy import stats
import plotly.express as px

# open csv and create df
patient_history_df = pd.read_csv('/Users/carrieminerich/Desktop/verana/patient_history.csv')

# lowercase column headers for easeier analysis
patient_history_df.columns = patient_history_df.columns.str.lower()

# matrix of # of cigs vs diabetes = 1
matrix = pd.crosstab(patient_history_df['cigsperday'], patient_history_df['diabetes'])
print(f"Matrix:{matrix}")

# chi-square test
chi2, p, _, _ = stats.chi2_contingency(matrix)

print(f"Chi-Square test for cigs per day and diabetes: {chi2}")
print(f"P-Value for cigs per day and diabetes: {p}")

# print results
if p < 0.05:
    print("The result is statistically significant. There is a relation between the number of cigarettes smoked per day and diabetes.")
else:
    print("The result is not statistically significant. There is no strong evidence of a relation between the number of cigarettes smoked per day and diabetes.")


# visualize results
fig = px.histogram(patient_history_df, x='cigsperday', color='diabetes', barmode='group',
                   title='Number of Cigarettes Smoked per Day vs. Prevalence of Diabetes',
                   labels={'cigsperday': 'Number of Cigarettes Smoked per Day', 'count': 'Count'},
                   category_orders={'diabetes': [0, 1]})

fig.update_layout(xaxis_title='Number of Cigarettes Smoked per Day',
                  yaxis_title='Total individual count in patient_history.csv',
                  legend_title='Diabetes: 1; No Diabetes:2')

fig.show()